In [ ]:
# This experiment compares the Toffoli implementation
# from https://arxiv.org/pdf/2501.02222v1
# with the default decomposition in Qiskit.

from qiskit import QuantumCircuit
import qiskit
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Operator
import math
from math import pi

from sqlalchemy.orm import joinedload
from sqlalchemy import select

from benchmarklib import BenchmarkDatabase
from benchmarklib.runners import BatchQueue

from experiments import ExperimentProblem, ExperimentTrial

TAG = "toffoli_comparison"

In [ ]:
service = QiskitRuntimeService()
backend = service.backend("ibm_rensselaer")

db = BenchmarkDatabase("experiments.db", ExperimentProblem, ExperimentTrial)

In [ ]:
#[db.delete_problem_instance(problem.id) for problem in db.query(select(ExperimentProblem).filter(ExperimentProblem.tag==TAG))]

In [ ]:
from qiskit.transpiler import CouplingMap
def _qiskit_default():
    # used this to produce the circuit for qiskit_default, and froze its instructions for reproducibility (since sabre can sometimes give different results, which relabele the qubits differently)
    qc = QuantumCircuit(3,3)
    qc.ccx(0, 1, 2)
    
    qc_transpiled = qiskit.transpile(
        qc,
        basis_gates=backend.configuration().basis_gates,
        coupling_map = CouplingMap([[0, 1], [1, 0], [1, 2], [2, 1]]),  # restrict use to just these qubits for consistency
        optimization_level=0,
        initial_layout=[0, 1, 2],
        routing_method='sabre',    # apply routing here to bring this on the same connectivity level as the other implementation
    )
    return qc_transpiled

def qiskit_default():
    qc = QuantumCircuit(3, 3)

    qc.rz(-pi/2, 0)
    qc.rz(-pi/2, 1)
    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 2)
    qc.ecr(1, 2)

    qc.x(1)
    qc.rz(-pi/2, 1)
    qc.rz(-pi/2, 1)
    qc.rz(-pi/4, 2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.ecr(1, 2)

    qc.sx(1)
    qc.rz(-pi/2, 2)
    qc.ecr(2, 1)

    qc.rz(-pi/2, 1)
    qc.sx(2)
    qc.ecr(1, 2)

    qc.ecr(0, 1)

    qc.x(0)
    qc.rz(-pi/2, 0)
    qc.rz(pi/4, 1)
    qc.rz(pi/2, 1)
    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.rz(pi/2, 1)
    qc.ecr(2, 1)

    qc.rz(-pi/4, 1)
    qc.rz(pi/2, 1)
    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.rz(pi/2, 1)
    qc.ecr(0, 1)

    qc.x(0)
    qc.rz(-pi/2, 0)
    qc.rz(pi/4, 1)
    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.rz(-pi/2, 1)
    qc.x(2)
    qc.rz(pi/4, 2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.ecr(1, 2)

    qc.sx(1)
    qc.rz(-pi/2, 2)
    qc.ecr(2, 1)

    qc.rz(-pi/2, 1)
    qc.sx(2)
    qc.ecr(1, 2)

    qc.ecr(0, 1)

    qc.x(0)
    qc.rz(pi/4, 0)
    qc.rz(-pi/2, 0)
    qc.rz(-pi/4, 1)
    qc.rz(pi/2, 1)
    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.rz(pi/2, 1)
    qc.ecr(0, 1)

    qc.x(0)

    return qc

def qiskit_optimized():
    qc = QuantumCircuit(3, 3)

    qc.rz(2.719050386660717, 2)
    qc.sx(2)
    qc.rz(0.7037464076776843, 1)
    qc.sx(1)
    qc.ecr(1, 2)

    qc.rz(-pi/2, 2)
    qc.sx(2)
    qc.rz(pi/2, 2)
    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(-pi/2, 1)
    qc.ecr(1, 2)

    qc.sx(2)
    qc.rz(-2.274542734472581, 2)
    qc.sx(2)
    qc.rz(-pi/2, 2)
    qc.rz(-2.3102287863342097, 1)
    qc.sx(1)
    qc.rz(-1.8649925135950234, 1)
    qc.sx(1)
    qc.rz(-1.878623226096499, 1)
    qc.rz(-pi/2, 0)
    qc.sx(0)
    qc.rz(-pi/2, 0)
    qc.ecr(1, 0)

    qc.rz(-pi/2, 1)
    qc.sx(1)
    qc.rz(-pi/4, 1)
    qc.sx(1)
    qc.ecr(1, 2)

    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.rz(3*pi/4, 2)
    qc.sx(2)
    qc.rz(-pi/2, 1)
    qc.sx(1)
    qc.rz(pi/4, 1)
    qc.sx(1)
    qc.ecr(1, 0)

    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(-pi/4, 1)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.ecr(1, 2)

    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.rz(-pi/2, 1)
    qc.sx(1)
    qc.rz(-pi, 1)
    qc.ecr(1, 2)

    qc.rz(-pi/2, 2)
    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.ecr(1, 2)

    qc.rz(pi/2, 2)
    qc.sx(2)
    qc.rz(pi/2, 1)
    qc.sx(1)
    qc.rz(-pi/2, 1)
    qc.rz(pi/2, 0)
    qc.sx(0)
    qc.rz(-1.1048615809796667, 0)
    qc.ecr(1, 0)

    qc.rz(-pi, 0)
    qc.sx(0)
    qc.rz(3*pi/4, 0)
    qc.sx(0)
    qc.ecr(1, 0)

    qc.x(1)
    qc.x(0)
    qc.rz(-1.890259744377115, 0)

    return qc

def toffoli_swapped_18():
    qc = QuantumCircuit(3)

    qc.rz(-pi/2, 0)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.rz(-pi/2, 2)
    qc.ecr(0, 1)

    qc.x(0)
    qc.rz(-pi/2, 0)
    qc.rz(3*pi/4, 1)
    qc.sx(1)
    qc.rz(-pi,1)
    qc.ecr(2, 1)

    qc.rz(-3*pi/4,1)
    qc.sx(1)
    qc.rz(-pi, 1)
    qc.x(2)
    qc.rz(-pi/2,2)
    qc.ecr(0,1)

    qc.rz(-3*pi/4,0)
    qc.sx(0)
    qc.rz(3*pi/4,1)
    qc.sx(1)
    qc.rz(-pi,1)
    qc.ecr(2,1)

    qc.rz(-3*pi/4,1)
    qc.sx(1)
    qc.rz(pi/2,1)
    qc.rz(-pi,2)
    qc.x(2)
    qc.ecr(2,1)

    qc.sx(1)
    qc.rz(-pi/2,1)
    qc.rz(-pi/2,2)
    qc.sx(2)
    qc.ecr(2,1)

    qc.rz(-pi,1)
    qc.sx(1)
    qc.rz(pi/2,1)
    qc.rz(pi/2,2)
    qc.sx(2)
    qc.ecr(2,1)
    
    qc.rz(pi/2,1)
    qc.sx(1)
    qc.rz(-pi/2,1)
    qc.ecr(0,1)

    qc.rz(-pi/2,0)
    qc.sx(0)
    qc.rz(pi/4,0)
    qc.sx(0)
    qc.rz(pi/2,1)
    qc.sx(1)
    qc.rz(-3*pi/4,1)
    qc.sx(1)
    qc.rz(pi/2,1)
    qc.ecr(0,1)

    qc.rz(pi/2,0)
    qc.sx(0)
    qc.rz(pi/2,0)
    qc.rz(-pi/2,1)
    qc.sx(1)
    qc.rz(pi/2,1)

    return qc

def toffoli_18():
    qc = QuantumCircuit(3)
    qc.swap(1, 2)

    qc.rz(-pi/2, 0)
    qc.sx(1)
    qc.rz(pi/2, 1)
    qc.rz(-pi/2, 2)
    qc.ecr(0, 1)

    qc.x(0)
    qc.rz(-pi/2, 0)
    qc.rz(3*pi/4, 1)
    qc.sx(1)
    qc.rz(-pi,1)
    qc.ecr(2, 1)

    qc.rz(-3*pi/4,1)
    qc.sx(1)
    qc.rz(-pi, 1)
    qc.x(2)
    qc.rz(-pi/2,2)
    qc.ecr(0,1)

    qc.rz(-3*pi/4,0)
    qc.sx(0)
    qc.rz(3*pi/4,1)
    qc.sx(1)
    qc.rz(-pi,1)
    qc.ecr(2,1)

    qc.rz(-3*pi/4,1)
    qc.sx(1)
    qc.rz(pi/2,1)
    qc.rz(-pi,2)
    qc.x(2)
    qc.ecr(2,1)

    qc.sx(1)
    qc.rz(-pi/2,1)
    qc.rz(-pi/2,2)
    qc.sx(2)
    qc.ecr(2,1)

    qc.rz(-pi,1)
    qc.sx(1)
    qc.rz(pi/2,1)
    qc.rz(pi/2,2)
    qc.sx(2)
    qc.ecr(2,1)
    
    qc.rz(pi/2,1)
    qc.sx(1)
    qc.rz(-pi/2,1)
    qc.ecr(0,1)

    qc.rz(-pi/2,0)
    qc.sx(0)
    qc.rz(pi/4,0)
    qc.sx(0)
    qc.rz(pi/2,1)
    qc.sx(1)
    qc.rz(-3*pi/4,1)
    qc.sx(1)
    qc.rz(pi/2,1)
    qc.ecr(0,1)

    qc.rz(pi/2,0)
    qc.sx(0)
    qc.rz(pi/2,0)
    qc.rz(-pi/2,1)
    qc.sx(1)
    qc.rz(pi/2,1)

    return qc



In [ ]:
def print_unitary(circuit, ignore_global_phase=True):
    op = Operator(circuit).data
    if ignore_global_phase:
        global_phase = op[0,0] / abs(op[0,0])
        op = op / global_phase
        
    op = op.round(3)

    for row in op:
        print(" ".join(f"{elem:7}" for elem in row))

In [ ]:
for circuit_factory in [qiskit_default, qiskit_optimized, toffoli_18, toffoli_swapped_18]:
    circuit = circuit_factory()
    print("-----", circuit_factory.__name__)
    print(circuit.count_ops())
    print_unitary(circuit)

In [ ]:
def create_trials(toffoli_circuit, name):
    trials = []
    # check if problem already existis
    existing = db.query(select(ExperimentProblem).where(ExperimentProblem.name == name, ExperimentProblem.tag == TAG))
    if len(existing) > 0:
        experiment = existing[0]
    else:
        experiment = ExperimentProblem(
            name=name,
            tag=TAG,
            n=3,
        )
    for i in range(2**3):
        input_state = format(i, '03b')
        qc = QuantumCircuit(3, 3)
        for qubit in range(3):
            if input_state[qubit] == '1':
                qc.x(qubit)
                
        qc.compose(toffoli_circuit, inplace=True)
        qc.measure(range(3), range(3))

        qc_final = transpile(
            qc,
            backend=backend,
            optimization_level=0,
            initial_layout=[0,1,2],
            routing_method='sabre', # although we have routing here, no routing should actually occur since the toffolis are routed
            basis_gates=None,
        )

        trials.append(
            ExperimentTrial(problem=experiment, circuit=qc_final, circuit_pretranspile=qc, extra_data={"input_state": input_state, "backend": backend.name})
        )

    return trials

    

In [ ]:
with BatchQueue(db, backend=backend, shots=4096) as q:
    for circuit, name in [
        (qiskit_default(), "qiskit_default"),
        (qiskit_optimized(), "qiskit_optimized"),
        (toffoli_18(), "toffoli_18"),
        (toffoli_swapped_18(), "toffoli_swapped_18"),
    ]:
        for trial in create_trials(circuit, name):
            q.enqueue(trial, trial.circuit, run_simulation=True)
    q.submit_tasks()
    

In [ ]:
await db.update_all_pending_results(service=service)

In [ ]:
trials =  db.query(
    select(ExperimentTrial).join(ExperimentTrial.problem).where(ExperimentProblem.tag==TAG).options(joinedload(ExperimentTrial.problem))
)

In [ ]:

from collections import defaultdict


for trial_name in ["qiskit_default", "qiskit_optimized", "toffoli_18", "toffoli_swapped_18"]:
    average_success_rate = 0.0
    count = 0
    trials =  db.query(
        select(ExperimentTrial).join(ExperimentTrial.problem).where(ExperimentProblem.tag==TAG, ExperimentProblem.name==trial_name).options(joinedload(ExperimentTrial.problem))
    )
    results = defaultdict(list)
    for trial in trials:
        results[trial.extra_data["input_state"]].append(trial.calculate_success_rate())
        average_success_rate += trial.calculate_success_rate()
        count += 1
    average_success_rate /= count
    print(f"Trial: {trial_name}, Average success rate: {average_success_rate:.2%}")
    print(results)


In [ ]:
db.query(
    select(ExperimentTrial).join(ExperimentTrial.problem).where(ExperimentProblem.tag==TAG, ExperimentProblem.name=="qiskit_optimized").options(joinedload(ExperimentTrial.problem))
)[0].circuit.depth()

In [ ]:
db.query(
    select(ExperimentTrial).join(ExperimentTrial.problem).where(ExperimentProblem.tag==TAG, ExperimentProblem.name=="toffoli_18").options(joinedload(ExperimentTrial.problem))
)[0].circuit.depth()

In [ ]:
for trial in db.query(
    select(ExperimentTrial).join(ExperimentTrial.problem).where(ExperimentProblem.tag==TAG, ExperimentProblem.name=="qiskit_default").options(joinedload(ExperimentTrial.problem))
)[-8:]:
    input_state = trial.extra_data["input_state"]
    success_rate = trial.calculate_success_rate()
    most_likely_output = max(trial.counts, key=trial.counts.get)
    most_likely_simulation_output = max(trial.simulation_counts, key=trial.simulation_counts.get)
    print(f"{input_state} -> {most_likely_output} (sim: {most_likely_simulation_output}), success rate: {success_rate:.2%}")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

def visualize_trial_comparison(trial_names, db, TAG):
    """
    Create a comparison visualization of success rates across multiple trial types.
    
    Parameters:
    -----------
    trial_names : list of str
        List of trial type names to compare
    db : database session
        Your database session object
    TAG : str
        The experiment tag to filter by
    """
    # Collect data for all trial types
    all_trial_data = {}
    
    for trial_name in trial_names:
        # Query the trials
        trials = db.query(
            select(ExperimentTrial).join(ExperimentTrial.problem).where(
                ExperimentProblem.tag==TAG, 
                ExperimentProblem.name==trial_name
            ).options(joinedload(ExperimentTrial.problem))
        )
        
        # Collect results by input state
        results = defaultdict(list)
        output_states = {}
        for trial in trials:
            input_state = trial.extra_data["input_state"]
            success_rate = trial.calculate_success_rate()
            results[input_state].append(success_rate)
            most_likely_output = max(trial.simulation_counts, key=trial.simulation_counts.get)
            output_states[input_state] = most_likely_output[::-1]
        
        # Calculate average success rate for each input state
        avg_success_rates = {}
        for input_state, rates in results.items():
            avg_success_rates[input_state] = np.mean(rates)
        
        # Store data
        all_trial_data[trial_name] = {
            'rates': avg_success_rates,
            'overall_avg': np.mean(list(avg_success_rates.values())),
            'output_states': output_states
        }
    
    # Create the comparison plot
    num_trials = len(trial_names)
    fig, axes = plt.subplots(num_trials, 1, figsize=(12, 4 * num_trials))
    
    # Handle single trial case
    if num_trials == 1:
        axes = [axes]
    
    # Plot each trial type
    for idx, trial_name in enumerate(trial_names):
        ax = axes[idx]
        data = all_trial_data[trial_name]
        
        # Create 2x4 grid data
        grid_data = np.zeros((2, 4))
        for i in range(8):
            input_state = f"{i:03b}"
            row = i // 4
            col = i % 4
            grid_data[row, col] = data['rates'].get(input_state, 0)
        
        # Create heatmap
        im = ax.imshow(grid_data, cmap='RdYlGn', vmin=0, vmax=1, aspect='auto')
        
        # Set ticks and labels
        ax.set_xticks(np.arange(4))
        ax.set_yticks(np.arange(2))
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        
        # Add input state labels and success rates in each cell
        for i in range(8):
            row = i // 4
            col = i % 4
            input_state = f"{i:03b}"
            output_state = data['output_states'].get(input_state, '---')
            success_rate = data['rates'].get(input_state, 0)
            
            # Input state label (top)
            ax.text(col, row - 0.25, f"|{input_state}⟩ → |{output_state}⟩", 
                    ha="center", va="center", color="black", fontsize=11, fontweight='bold')
            
            # Success rate (bottom)
            color = "white" if success_rate < 0.5 else "black"
            ax.text(col, row + 0.25, f"{success_rate:.1%}", 
                    ha="center", va="center", color=color, fontsize=13, fontweight='bold')
        
        # Set title with overall average
        ax.set_title(f'{trial_name} (Overall Avg: {data["overall_avg"]:.2%})', 
                     fontsize=13, fontweight='bold', pad=15)
        
        # Remove tick marks
        ax.tick_params(length=0)
        
        # Add grid
        ax.set_xticks(np.arange(4) - 0.5, minor=True)
        ax.set_yticks(np.arange(2) - 0.5, minor=True)
        ax.grid(which="minor", color="gray", linestyle='-', linewidth=2)
    
    # Add horizontal colorbar at the bottom
    fig.subplots_adjust(bottom=0.12)
    cbar_ax = fig.add_axes([0.25, 0.05, 0.5, 0.02])
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Success Rate', fontsize=12)
    
    plt.suptitle('Toffoli Gate Implementation Comparison', 
                 fontsize=16, fontweight='bold')
    plt.tight_layout(rect=[0, 0.08, 1, 0.96])
    return fig, axes


In [ ]:
len(db.query(select(ExperimentTrial).join(ExperimentTrial.problem).where(ExperimentProblem.name=="qiskit_default")))

In [ ]:
fig, axes = visualize_trial_comparison(
    ["qiskit_default", "qiskit_optimized", "toffoli_18", "toffoli_swapped_18"], 
    db, 
    TAG
)
plt.savefig("toffoli_comparison.png", dpi=300, bbox_inches='tight')
plt.show()